In [1]:
#Import library
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [3]:
#define split function to convert sequence into X and y
def split_sequence(sequence, n_steps_in, n_steps_out):
  X, y = list (), list ()
  for i in range(len(sequence)):
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out
    if out_end_ix > len(sequence):
      break
    seq_x, seq_y = sequence[i: end_ix, :-1], sequence[end_ix: out_end_ix, -1]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [4]:
#Define sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i] + in_seq2[i] for i in range (len(in_seq1))])


In [5]:
#Convert into rows and column 
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [6]:
#Horizontal stack
dataset = hstack((in_seq1, in_seq2, out_seq))

In [7]:
#define n_steps_in and n_steps_out
n_steps_in, n_steps_out = 3,2

In [8]:
#Split into X and y dataset
X, y = split_sequence(dataset, n_steps_in, n_steps_out)

In [9]:
#Explore X and y dataset
print(X.shape, y.shape)
for i in range(len(X)):
  print(X[i], y[i])

(5, 3, 2) (5, 2)
[[10 15]
 [20 25]
 [30 35]] [ 85 105]
[[20 25]
 [30 35]
 [40 45]] [105 125]
[[30 35]
 [40 45]
 [50 55]] [125 145]
[[40 45]
 [50 55]
 [60 65]] [145 165]
[[50 55]
 [60 65]
 [70 75]] [165 185]


In [10]:
#Define n_feature
n_feature = X.shape[2]

In [11]:
#Reshape into 3D structure [sample, n_steps, n_feature]
X = X.reshape((X.shape[0], n_steps_in, n_feature))

In [12]:
#Define model
model = Sequential()
model.add(Conv1D(filters= 64, kernel_size= 2, activation= 'relu', input_shape = (n_steps_in, n_feature)))
model.add(MaxPooling1D(pool_size= 2))
model.add(Flatten())
model.add(Dense(50, activation= 'relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer = 'adam', loss = 'mse')

In [13]:
#Summary model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 2, 64)             320       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 50)                3250      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 102       
Total params: 3,672
Trainable params: 3,672
Non-trainable params: 0
_________________________________________________________________


In [14]:
#Fit model
model.fit(X, y, epochs= 2000, verbose= 0)

In [15]:
#Demonstrate prediction
x_input = array([[50, 55], [60, 65], [70, 75]])
x_input = x_input.reshape((1, n_steps_in, n_feature))
y_hat = model.predict(x_input, verbose=0)
print(y_hat)

[[164.98882 185.02072]]
